##Generate Threat Types

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import pandas as pd
import json, csv, logging, os, sys, re, requests, zipfile
from os import listdir
from os.path import join, isfile
from sklearn.model_selection import train_test_split

In [41]:
def get_nvd_data():
    r = requests.get('https://nvd.nist.gov/vuln/data-feeds#JSON_FEED')
    for filename in re.findall("nvdcve-1.1-[0-9]*\.json\.zip",r.text):
        print(filename)
        r_file = requests.get("https://nvd.nist.gov/feeds/json/cve/1.1/" + filename, stream=True)
        with open("/content/drive/MyDrive/Python_Training/Final_Project/zip/" + filename, 'wb') as f:
            for chunk in r_file:
                f.write(chunk)


def unzip_data():
    files = [f for f in listdir("/content/drive/MyDrive/Python_Training/Final_Project/zip/") if isfile(join("/content/drive/MyDrive/Python_Training/Final_Project/zip/", f))]
    files.sort()
    for file in files:
        print("Opening: " + file)
        archive = zipfile.ZipFile(join("/content/drive/MyDrive/Python_Training/Final_Project/zip/", file), 'r')
        with archive as f:
            f.extractall('/content/drive/MyDrive/Python_Training/Final_Project/jsonFile')

In [42]:
get_nvd_data()
unzip_data()

nvdcve-1.1-2023.json.zip
nvdcve-1.1-2022.json.zip
nvdcve-1.1-2021.json.zip
nvdcve-1.1-2020.json.zip
nvdcve-1.1-2019.json.zip
nvdcve-1.1-2018.json.zip
nvdcve-1.1-2017.json.zip
nvdcve-1.1-2016.json.zip
nvdcve-1.1-2015.json.zip
nvdcve-1.1-2014.json.zip
nvdcve-1.1-2013.json.zip
nvdcve-1.1-2012.json.zip
nvdcve-1.1-2011.json.zip
nvdcve-1.1-2010.json.zip
nvdcve-1.1-2009.json.zip
nvdcve-1.1-2008.json.zip
nvdcve-1.1-2007.json.zip
nvdcve-1.1-2006.json.zip
nvdcve-1.1-2005.json.zip
nvdcve-1.1-2004.json.zip
nvdcve-1.1-2003.json.zip
nvdcve-1.1-2002.json.zip
Opening: nvdcve-1.1-2002.json.zip
Opening: nvdcve-1.1-2003.json.zip
Opening: nvdcve-1.1-2004.json.zip
Opening: nvdcve-1.1-2005.json.zip
Opening: nvdcve-1.1-2006.json.zip
Opening: nvdcve-1.1-2007.json.zip
Opening: nvdcve-1.1-2008.json.zip
Opening: nvdcve-1.1-2009.json.zip
Opening: nvdcve-1.1-2010.json.zip
Opening: nvdcve-1.1-2011.json.zip
Opening: nvdcve-1.1-2012.json.zip
Opening: nvdcve-1.1-2013.json.zip
Opening: nvdcve-1.1-2014.json.zip
Opening:

In [45]:
def create_nvd_dict(year):
    filename = join("/content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-" + str(year) + ".json")
    print("Opening: " + filename)
    with open(filename, encoding="utf8") as json_file:
        cve_dict = json.load(json_file)
    return(cve_dict)


def generate_threattypes():
    list = listdir("/content/drive/MyDrive/Python_Training/Final_Project/jsonFile/")
    number_files = len(list)
    for year in range(2002,2002 + number_files):
        year_in_string = str(year)
        cve_dict = create_nvd_dict(year)
        fileName = 'NVD_'+ year_in_string + '_threat_train.csv'
        folder_path = '/content/drive/MyDrive/Python_Training/Final_Project/nvd'
        file_path = os.path.join(folder_path, fileName)
        with open('/content/drive/MyDrive/Python_Training/Final_Project/nvd/' + fileName, 'w', newline='') as f_output:
            print(fileName)
            csv_output = csv.writer(f_output)
            csv_output.writerow(['id', 'report','memc', 'bypass', 'csrf', 'dirtra', 'dos', 'execution', 'fileinc', 'gainpre', 'httprs', 'infor', 'overflow',
            'sqli', 'xss'])

            for item in cve_dict['CVE_Items']:
                cve_id = item['cve']['CVE_data_meta']['ID']
                report = item['cve']['description']['description_data'][0]['value']
                if not report.find("REJECT"):
                    continue

                memc = 0
                bypass = 0
                csrf = 0
                dirtra = 0
                dos = 0
                execution = 0
                fileinc = 0
                gainpre = 0
                httprs = 0
                infor = 0
                overflow = 0
                sqli = 0
                xss = 0

                name_cat = ['memc', 'bypass', 'csrf', 'dirtra', 'dos', 'execution', 'fileinc', 'gainpre', 'httprs', 'infor', 'overflow','sqli', 'xss']
                threat_list = [memc, bypass, csrf, dirtra, dos, execution, fileinc, gainpre, httprs, infor, overflow, sqli, xss]
                for cat_idx in range(13):
                    fobj = open('/content/drive/MyDrive/Python_Training/Final_Project/cveIDThreatType/cve_id' + name_cat[cat_idx] + '.txt', 'r')
                    text = fobj.read().strip().split()
                    try:
                        s = cve_id
                        if s == "":
                            continue
                        elif s in text:
                            threat_list[cat_idx]=1
                        else:
                            threat_list[cat_idx]=0

                    except Exception as e:
                        print(e)

                memc = threat_list[0]
                bypass = threat_list[1]
                csrf = threat_list[2]
                dirtra = threat_list[3]
                dos = threat_list[4]
                execution = threat_list[5]
                fileinc = threat_list[6]
                gainpre = threat_list[7]
                httprs = threat_list[8]
                infor = threat_list[9]
                overflow = threat_list[10]
                sqli = threat_list[11]
                xss = threat_list[12]

                csv_output.writerow([cve_id, report,memc, bypass, csrf, dirtra, dos, execution, fileinc, gainpre, httprs, infor, overflow, sqli, xss])

In [46]:
generate_threattypes()

Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2002.json
NVD_2002_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2003.json
NVD_2003_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2004.json
NVD_2004_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2005.json
NVD_2005_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2006.json
NVD_2006_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2007.json
NVD_2007_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2008.json
NVD_2008_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project/jsonFile/nvdcve-1.1-2009.json
NVD_2009_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Pr

##Combined File

In [47]:
def generate_CombinedFile():
    folder_path = "/content/drive/MyDrive/Python_Training/Final_Project/nvd/"
    file_list = listdir(folder_path)
    number_files = len(file_list)
    dict_of_reports = {}
    dict = []
    for year in range(2002, 2002 + number_files):
        year_in_string = str(year)
        file_name = 'NVD_' + year_in_string + '_threat_train.csv'
        file_path = folder_path + file_name

        dict_of_reports[year_in_string] = pd.read_csv(file_path)
        dict.append(dict_of_reports[year_in_string])

    df = pd.concat(dict, ignore_index=True)
    df = df[~df['report'].str.contains('REJECT')]
    rowsums=df.iloc[:,2:].sum(axis=1)
    x=rowsums.value_counts()
    df['clean']=(rowsums==0)
    indexNames = df[ df['clean'] == True].index

    df.drop(indexNames, inplace=True)
    df.drop('clean', inplace=True, axis=1)
    output_file_path = "/content/drive/MyDrive/Python_Training/Final_Project/combined_data.csv"
    df.to_csv(output_file_path, index=False)

    return df

In [48]:
df = generate_CombinedFile()

In [49]:
df

,id,report,memc,bypass,csrf,dirtra,dos,execution,fileinc,gainpre,httprs,infor,overflow,sqli,xss
0,CVE-1999-0001,ip_input.c in BSD-derived TCP/IP implementatio...,0,0,0,0,1,0,0,0,0,0,0,0,0
1,CVE-1999-0002,Buffer overflow in NFS mountd gives root acces...,0,0,0,0,0,0,0,0,0,0,1,0,0
2,CVE-1999-0003,Execute commands as root via buffer overflow i...,0,0,0,0,0,0,0,0,0,0,1,0,0
3,CVE-1999-0004,"MIME buffer overflow in email clients, e.g. So...",0,0,0,0,0,0,0,0,0,0,1,0,0
4,CVE-1999-0005,Arbitrary command execution via IMAP buffer ov...,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183597,CVE-2021-43141,Cross Site Scripting (XSS) vulnerability exist...,0,0,0,0,0,0,0,0,0,0,0,0,1
183701,CVE-2021-43264,"In Mahara before 20.04.5, 20.10.3, 21.04.2, an...",0,1,0,1,0,0,0,0,0,0,0,0,0
183702,CVE-2021-43265,"In Mahara before 20.04.5, 20.10.3, 21.04.2, an...",0,0,0,0,0,0,0,0,0,0,0,0,1
183703,CVE-2021-43266,"In Mahara before 20.04.5, 20.10.3, 21.04.2, an...",0,0,0,0,0,1,0,0,0,0,0,0,0
